In [195]:
import requests
from bs4 import BeautifulSoup
import re
import json
import collections
import pandas as pd

In [2]:
html = requests.get("https://www.fantasyfootballscout.co.uk/team-news/").text
soup = BeautifulSoup(html, "html.parser")


In [3]:
raw = soup.find_all("div", {"class": re.compile('formation.*')})
teams = [x.text.split('Next')[0] for x in soup.find_all('header')[-20:]]
lineups = [x.text.split('  ')[1:] for x in raw]

In [4]:
xi = {team: lineup for team, lineup in zip(teams, lineups)}

In [5]:
html = requests.get("https://understat.com/league/EPL").text
soup = BeautifulSoup(html, "html.parser")

In [203]:
pscript = soup.find_all('script')[3].string

In [204]:
sidx = pscript.index("parse") + 7
eidx = pscript.index(");") - 1

In [205]:
playerdata = json.loads(pscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [211]:
fd = collections.defaultdict(list)

for d in playerdata:
    for key, value in d.items():
        fd[key].append(value)
players = pd.DataFrame(fd)

In [242]:
teamscript = soup.find_all('script')[2].string
sidx = b.index("parse") + 7
eidx = b.index(")") - 1

In [243]:
teamdata = json.loads(teamscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [244]:
len(teamdata)

20

In [246]:
players

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,1250,Mohamed Salah,20,1786,16,15.608175069093704,9,6.22314802929759,80,40,1,0,F,Liverpool,14,13.324668571352959,22.21259716153145,7.717494197189808
1,6854,Diogo Jota,20,1487,10,12.501362554728985,1,3.721017401665449,58,23,1,0,F S,Liverpool,10,12.501362554728985,18.975039415061474,5.526300131343305
2,755,Jamie Vardy,16,1325,9,6.063936270773411,1,1.183594960719347,36,14,2,0,F S,Leicester,9,6.063936270773411,7.677956260740757,1.25945988856256
3,453,Son Heung-Min,17,1499,8,6.958429858088493,3,3.3822450675070286,44,33,0,0,F M,Tottenham,8,6.958429858088493,11.58835968375206,3.5982557721436024
4,531,Michail Antonio,21,1842,8,9.476286508142948,6,4.611624114215374,56,36,4,1,F S,West Ham,8,8.715117610991001,16.13105996698141,6.068427070975304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,10166,Kasey McAteer,1,1,0,0,0,0,0,0,0,0,S,Leicester,0,0,0,0
491,10172,Jonathan Rowe,2,29,0,0,0,0.08120320737361908,0,1,0,0,S,Norwich,0,0,0.16017208993434906,0.16017208993434906
492,10290,Edo Kayembe,1,90,0,0.04149774834513664,0,0.298225075006485,2,2,0,0,M,Watford,0,0.04149774834513664,0.46901270747184753,0.187638059258461
493,10291,Vitalii Mykolenko,1,90,0,0,0,0,0,0,0,0,D,Everton,0,0,0.1117921695113182,0.1117921695113182
